# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [148]:
import numpy as np
import scipy as sc
import random as rn
import urllib.request


In [249]:
def read_col_file(filename): 
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
           # print("\nСчитанные вершины и ребра:",vertexes,edges,'\n')
        return (vertices, edges)

In [250]:
def fac(n):
     if n == 0:
          return 1
     return fac(n-1) * n

In [251]:
def change(A:set,B:set, e1, e2:int):
    A.remove(e1)
    B.remove(e2)
    A.add(e2)
    B.add(e1)

In [254]:
def basic_local_search(graph): #принимает граф в формате read_col_file 
    vertexes, edges = graph
    tmp = 0
    n = len(vertexes)
    L = set(rn.sample(list(vertexes), len(vertexes) // 2)) # случайное деление графа на две части
    print(L)
    R = vertexes - L
    #print (L, R)
    for e in edges: # edges из второй ячейки
        a,b = e
        if (a in L and b in R) or (a in L and b in R):
           tmp += 1
    i = 0
    permutations = fac(n)
    for i in range(permutations):
        tmpL, tmpR = [],[] #заведем массивы для будущих минимальных L и R и рассмотрим перестановки вершин:
        for a in L:
            for b in R:
                ttmpL, ttmpR = L.copy(), R.copy()# чтобы не "побить" оригиналы
                ttmpR.remove(b)
                ttmpL.remove(a)
                ttmpL.add(a)
                ttmpR.add(b)
                Ctmp = 0
                for ed in edges:
                    c,d = ed
                    if (c in ttmpR and d in ttmpL)or(c in ttmpL and d in ttmpR):
                        Ctmp += 1
                if Ctmp < tmp:
                    tmpL, tmpR = ttmpL, ttmpR
                    tmp = Ctmp
        if len(tmpL) == 0:
            return (L, R)
        else:
            if (len(ttmpL) > len(ttmpR)):
                    L,R = ttmpR, ttmpL
                    tmp = Ctmp
        # return vprime
    pass

In [255]:

for i in range(3,8):
    url_instances = 'http://mat.gsia.cmu.edu/COLOR/instances/myciel'+str(i)+'.col'
    print("Testing file number ", i, ":\n")
    print (url_instances)
    with open('myceil'+str(i)+'col','w') as file:
        response = urllib.request.urlopen(url_instances)
        print("response",response.read().decode('utf-8'),file=file)
    vertexes, edges = read_col_file(str('myceil'+str(i)+'col'))  #пробегаемся по файлам и читаем их
    print (vertexes, edges)
    c = (TestL, TestR) = basic_local_search((vertexes,edges))
    print ("\nВычисленное разбиение:",c,"\n")
    

Testing file number  3 :

http://mat.gsia.cmu.edu/COLOR/instances/myciel3.col
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11} {(1, 2), (4, 10), (5, 9), (2, 6), (10, 11), (4, 6), (4, 5), (2, 8), (3, 10), (6, 11), (8, 11), (1, 4), (9, 11), (2, 3), (1, 9), (7, 11), (1, 7), (3, 7), (5, 8), (3, 5)}
{3, 6, 7, 9, 10}

Вычисленное разбиение: ({3, 6, 7, 9, 10}, {1, 2, 4, 5, 8, 11}) 

Testing file number  4 :

http://mat.gsia.cmu.edu/COLOR/instances/myciel4.col
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23} {(5, 9), (8, 13), (10, 11), (22, 23), (7, 12), (2, 8), (9, 22), (1, 15), (3, 21), (5, 19), (10, 15), (4, 12), (2, 12), (3, 7), (7, 11), (11, 19), (1, 20), (5, 8), (1, 2), (18, 23), (2, 19), (15, 23), (3, 10), (9, 11), (6, 22), (3, 16), (17, 23), (11, 18), (8, 16), (6, 15), (10, 22), (14, 23), (4, 10), (7, 14), (2, 6), (3, 13), (4, 5), (1, 13), (6, 11), (8, 11), (21, 23), (1, 4), (4, 16), (16, 23), (9, 16), (5, 15), (2, 3), (1, 9), (11, 21), (7, 22), (2, 14), (2, 17), 

In [248]:
import matplotlib as pt